In [3]:
!pip install isodate

In [ ]:
from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime, timedelta

api_key = ''

channel_id = 'UC7cs8q-gJRlGwj4A8OmCmXg'

youtube = build("youtube", "v3", developerKey=api_key)



def get_channel_metadata(youtube, channel_id):
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=channel_id
    )
    response = request.execute()
    item = response['items'][0]

    metadata = {
        "Channel_name": item["snippet"]["title"],
        "Subscribers": item["statistics"]["subscriberCount"],
        "Views": item["statistics"]["viewCount"],
        "PublishedAt": item["snippet"]["publishedAt"],
        "Country": item.get("snippet", {}).get("country", "N/A"),
        "Total_videos": item["statistics"]["videoCount"],
        "playlist_id": item["contentDetails"]["relatedPlaylists"]["uploads"]
    }
    return metadata

channel_metadata = get_channel_metadata(youtube, channel_id)
channel_df = pd.DataFrame([channel_metadata])

channel_df

channel_df.to_csv("channel_metadata.csv", index=False)





from datetime import datetime, timedelta
import isodate

def get_all_videos_with_stats(youtube, playlist_id):
    videos = []
    next_page_token = None

    while True:
        request = youtube.playlistItems().list(
            part="snippet",
            playlistId=playlist_id,
            maxResults=50,
            pageToken=next_page_token
        )
        response = request.execute()

        for item in response['items']:
            published_at = item["snippet"]["publishedAt"]
            video_id = item["snippet"]["resourceId"]["videoId"]
            video_title = item["snippet"]["title"]

            stats_response = youtube.videos().list(
                part="statistics,contentDetails",
                id=video_id
            ).execute()

            video_data = stats_response['items'][0]
            stats = video_data['statistics']
            duration_iso = video_data['contentDetails']['duration']
            duration_seconds = isodate.parse_duration(duration_iso).total_seconds()

            videos.append({
                "video_id": video_id,
                "video_title": video_title,
                "published_at": published_at,
                "view_count": int(stats.get("viewCount", 0)),
                "like_count": int(stats.get("likeCount", 0)),
                "comment_count": int(stats.get("commentCount", 0)),
                "duration_seconds": duration_seconds
            })

        next_page_token = response.get("nextPageToken")
        if not next_page_token:
            break

    return pd.DataFrame(videos)

playlist_id = channel_df["playlist_id"].iloc[0]
df_videos = get_all_videos_with_stats(youtube, playlist_id)

df_videos.head(5)

df_videos.to_csv("videos_data.csv", index=False)



def get_video_comments(youtube, video_id, max_comments=100):
    comments = []
    next_page_token = None
    while True:
        request = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            maxResults=100,
            pageToken=next_page_token,
            textFormat="plainText"
        )
        response = request.execute()

        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            comments.append({
                'video_id': video_id,
                'comment': comment
            })
            if len(comments) >= max_comments:
                break

        next_page_token = response.get('nextPageToken')
        if not next_page_token or len(comments) >= max_comments:
            break
    return comments

all_comments = []
for vid in df_videos['video_id']:
    comments = get_video_comments(youtube, vid, max_comments=500)
    all_comments.extend(comments)

df_comments = pd.DataFrame(all_comments)
df_comments.to_csv("comments.csv", index=False)
